In [1]:
import simpy
import numpy as np

In [2]:
# 시뮬레이션 파라미터 설정
num_vehicles = 10
num_subframes = 10  # 서브프레임 수
num_subchannels = 2  # 서브채널 수
vehicle_speed = 140 / 3.6  # km/h를 m/s로 변환
transmission_power = 23  # dBm
cam_size = 190  # bytes
cam_periodicity = 0.1  # seconds
subframe_duration = 0.001  # seconds
doca_length = 500  # meters
env = simpy.Environment()

In [4]:
# 전송 및 수신 카운터
total_packets_sent = 0
successful_packets_received = 0

In [5]:
# 스케줄러 클래스
class Scheduler:
    def __init__(self, env, num_subframes, num_subchannels):
        self.env = env
        self.subchannels = [[simpy.Resource(env) for _ in range(num_subchannels)] for _ in range(num_subframes)]

    def allocate_subchannel(self):
        # 각 서브프레임에서 사용 가능한 서브채널 찾기
        for subframe in self.subchannels:
            for subchannel in subframe:
                if not subchannel.users:  # 사용 가능한 경우
                    return subchannel
        return None  # 모든 서브채널이 사용 중

In [6]:
# 차량 클래스
class Vehicle:
    def __init__(self, env, id, speed, scheduler):
        self.env = env
        self.id = id
        self.speed = speed
        self.scheduler = scheduler
        self.action = env.process(self.run())

    def run(self):
        global total_packets_sent, successful_packets_received
        while True:
            subchannel = self.scheduler.allocate_subchannel()
            if subchannel is not None:
                total_packets_sent += 1
                yield self.env.timeout(cam_periodicity)
                with subchannel.request() as req:
                    yield req
                    yield self.env.timeout(subframe_duration)
                    successful_packets_received += 1
                    print(f"Time {self.env.now}: Vehicle {self.id} transmitted a CAM on subchannel.")
            else:
                yield self.env.timeout(cam_periodicity)

In [8]:
#스케줄러 인스턴스 생성
scheduler = Scheduler(env, num_subframes, num_subchannels)

In [9]:
# 차량 인스턴스 생성 및 시뮬레이션 실행
vehicles = [Vehicle(env, i, vehicle_speed, scheduler) for i in range(num_vehicles)]
env.run(until=10)  # 10초 동안 실행

Time 0.101: Vehicle 0 transmitted a CAM on subchannel.
Time 0.10200000000000001: Vehicle 1 transmitted a CAM on subchannel.
Time 0.10300000000000001: Vehicle 2 transmitted a CAM on subchannel.
Time 0.10400000000000001: Vehicle 3 transmitted a CAM on subchannel.
Time 0.10500000000000001: Vehicle 4 transmitted a CAM on subchannel.
Time 0.10600000000000001: Vehicle 5 transmitted a CAM on subchannel.
Time 0.10700000000000001: Vehicle 6 transmitted a CAM on subchannel.
Time 0.10800000000000001: Vehicle 7 transmitted a CAM on subchannel.
Time 0.10900000000000001: Vehicle 8 transmitted a CAM on subchannel.
Time 0.11000000000000001: Vehicle 9 transmitted a CAM on subchannel.
Time 0.202: Vehicle 0 transmitted a CAM on subchannel.
Time 0.203: Vehicle 1 transmitted a CAM on subchannel.
Time 0.20400000000000001: Vehicle 2 transmitted a CAM on subchannel.
Time 0.20500000000000002: Vehicle 3 transmitted a CAM on subchannel.
Time 0.20600000000000002: Vehicle 4 transmitted a CAM on subchannel.
Time 0.

In [10]:
# PRR 계산
prr = successful_packets_received / total_packets_sent if total_packets_sent > 0 else 0
print(f"PRR: {prr:.2f}")

PRR: 0.99
